我們從編寫一個最簡單的 多層感知器 （Multilayer Perceptron, MLP），或者說 “多層全連接神經網路” 開始，介紹 TensorFlow 的模型編寫方式。在這一部分，我們依次進行以下步驟：

使用 tf.keras.datasets 獲得資料集並預處理

使用 tf.keras.Model 和 tf.keras.layers 建構模型

建構模型訓練流程，使用 tf.keras.losses 計算損失函數，並使用 tf.keras.optimizer 優化模型

構建模型評估流程，使用 tf.keras.metrics 計算評量指標

**資料獲取及預處理： tf.keras.datasets**

先進行預備工作，實現一個簡單的 MNISTLoader 類來讀取 MNIST 資料集資料。這裡使用了 tf.keras.datasets 快速載入 MNIST 資料集。

In [ ]:
import tensorflow as tf
import numpy as np

class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

**模型的建構： tf.keras.Model 和 tf.keras.layers**

多層感知器的模型類別實現與上面的線性模型類似，使用 tf.keras.Model 和 tf.keras.layers 建構，所不同的地方在於層數增加了（顧名思義，“多層” 感知器），以及引入了非線性激活函數（這裡使用了 ReLU 函數 ， 即下方的 activation=tf.nn.relu ）。該模型輸入一個向量（比如這裡是拉直的 1×784 手寫體數字圖片），輸出 10 維的向量，分別代表這張圖片屬於 0 到 9 的機率。

In [ ]:
class MLP(tf.keras.Model):
  def __init__(self):
      super().__init__()
      self.flatten = tf.keras.layers.Flatten()    # Flatten層將除第一維（batch_size）以外的維度展平
      self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
      self.dense2 = tf.keras.layers.Dense(units=10)

  def call(self, inputs):         # [batch_size, 28, 28, 1]
      x = self.flatten(inputs)    # [batch_size, 784]
      x = self.dense1(x)          # [batch_size, 100]
      x = self.dense2(x)          # [batch_size, 10]
      output = tf.nn.softmax(x)
      return output

**模型的訓練： tf.keras.losses 和 tf.keras.optimizer**

定義一些模型超參數 (Hyperparameters)：

In [ ]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

實例化模型和資料讀取類，並實例化一個 tf.keras.optimizer 的優化器（這裡使用常用的 Adam 優化器）：

In [ ]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

11493376/11490434 [==============================] - 0s 0us/step


然後疊代進行以下步驟：

從 DataLoader 中隨機取一批訓練資料；

將這批資料送入模型，計算出模型的預測值；

將模型預測值與真實值進行比較，計算損失函數（loss）。這里使用 tf.keras.losses 中的交叉熵函數作為損失函數；

計算損失函數關於模型變數的導數；

將求出的導數值傳入優化器，使用優化器的 apply_gradients 方法更新模型參數以最小化損失函數（優化器的詳細使用方法見 前章 ）。

具體代碼實現如下：

In [ ]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

串流輸出內容已截斷至最後 5000 行。
batch 1001: loss 0.115957
batch 1002: loss 0.329780
batch 1003: loss 0.177451
batch 1004: loss 0.056999
batch 1005: loss 0.132606
batch 1006: loss 0.093699
batch 1007: loss 0.047782
batch 1008: loss 0.110802
batch 1009: loss 0.182394
batch 1010: loss 0.312375
batch 1011: loss 0.255396
batch 1012: loss 0.352247
batch 1013: loss 0.194108
batch 1014: loss 0.055381
batch 1015: loss 0.170574
batch 1016: loss 0.317325
batch 1017: loss 0.280596
batch 1018: loss 0.110626
batch 1019: loss 0.211503
batch 1020: loss 0.163204
batch 1021: loss 0.203231
batch 1022: loss 0.165183
batch 1023: loss 0.057147
batch 1024: loss 0.266518
batch 1025: loss 0.170883
batch 1026: loss 0.217634
batch 1027: loss 0.109817
batch 1028: loss 0.227527
batch 1029: loss 0.145468
batch 1030: loss 0.155538
batch 1031: loss 0.194013
batch 1032: loss 0.081543
batch 1033: loss 0.100293
batch 1034: loss 0.117673
batch 1035: loss 0.245565
batch 1036: loss 0.113264
batch 1037: loss 0.138791
batch 1038: loss 

**模型的評估： tf.keras.metrics**

最後，我們使用測試集評估模型的性能。這裡，我們使用 tf.keras.metrics 中的 SparseCategoricalAccuracy 評量器來評估模型在測試集上的性能，該評量器能夠對模型預測的結果與真實結果進行比較，並輸出預測正確的樣本數占總樣本數的比例。我們疊代測試資料集，每次通過 update_state() 方法向評量器輸入兩個參數： y_pred 和 y_true ，即模型預測出的結果和真實結果。評量器具有內部變數來保存當前評估指標相關的參數數值（例如當前已傳入的累計樣本數和當前預測正確的樣本數）。疊代結束後，我們使用 result() 方法輸出最終的評量指標值（預測正確的樣本數占總樣本數的比例）。

In [ ]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.975300
